In [10]:
# This code does the following: 
# Reads all .csv files in a given folder
# Reads the following data: Run#, Ewatts, Awatts, etc

# Csv file has the following format: 
#****************************************************
# 040686
# freq 9.040000
# Run#,Ewatts,Awatts,Eff,Pwatts,G3eff,RealImp,ImgImp
# 1, 41.5, 15.8, 38.1, 47.7, 32.7, 39.2,-24.1
# 2, 43.6, 16.4, 37.6, 50.2, 32.3, 39.2,-24.1
# 3, 44.1, 16.6, 37.6, 50.9, 32.2, 39.2,-24.0
# EffDiff,Apower,Eff,G3eff
# 1,1,1,1
# Date: 16-Oct-2025 12:45:30
# Lot: M4981

# [RFB]
# Filename:	O:\Transducers\Transducer Efficiency Testing\FXT-0304 RFB-2000 Transducer Efficiency Measurement System\M4981\040686-3.RFB
# Software Version	2.0.9.0
# Controller Serial Number	1155
# Tank Serial Number	1146
# Sensor Serial Number	0217
# Firmware Version	1.0.10

# [DUT]
# Model	Paradise
# Serial	Various
# ExcModel	FXT-0160
# ExcSerial	RM0931
# ExcMode	CW
# ExcLevel	41.5
# NomFreq	9

#****************************************************

import pandas as pd
import os

def find_csv_files(data_folder):
    # finds all csv files in given directly by looking through all files and subdirectories
    import os
    csv_files = []
    for root, dirs, files in os.walk(data_folder):
        for file in files:
            if file.endswith('.csv') and 'octiv' not in file.lower():
                csv_files.append(os.path.join(root, file))
    return csv_files

def read_sensor_data(file_path):
    # This function reads the sensor data from each file and returns a DataFrame

    with open(file_path, 'r') as file:

        print(file_path)
        lines = file.readlines()
    
    sn= lines[0].strip()  # Serial Number from first line
    freq= lines[1].strip().split()[1]  # Frequency from second line

    #print(f"Serial Number: {sn}, Frequency: {freq}")

    # Read RFB System details and DUT details
    rfb_details = {}
    dut_details = {}
    current_section = None
    for line in lines:
        line = line.strip()
        if "Date:" in line:
            test_time = line.split("Date:")[1].strip()
        if("Lot:" in line):
            lot_number = line.split("Lot:")[1].strip()

        if line == "[RFB]":
            current_section = "RFB"
            continue
        elif line == "[DUT]":
            current_section = "DUT"
            continue
        elif line.startswith("[") and line.endswith("]"):
            current_section = None
            continue
        
        if current_section == "RFB" and '\t' in line:
            key, value = line.split('\t', 1)
            rfb_details[key.strip()] = value.strip()
        elif current_section == "DUT" and '\t' in line:
            key, value = line.split('\t', 1)
            dut_details[key.strip()] = value.strip()



    # If you see Run# line, read next 3 lines of data with columns: Run#,Ewatts,Awatts,Eff,Pwatts,G3eff,RealImp,ImgImp
    for i, line in enumerate(lines):
        if "Run#" in line:
            run_data_start_index = i
            print(f"Found Run# data at line {i}")
            break
    else:
        run_data_start_index = None # No Run# found
    
    if run_data_start_index:
        run_data_lines = lines[run_data_start_index:run_data_start_index + 4]
        run_data = pd.DataFrame([x.strip().split(',') for x in run_data_lines[1:]], columns=run_data_lines[0].strip().split(','))
        run_data = run_data.apply(pd.to_numeric, errors='ignore')
    else:
        run_data = pd.DataFrame()  # Empty DataFrame if no Run# data found

    return sn, freq, test_time, lot_number, rfb_details, dut_details, run_data



if __name__ == "__main__":
    data_folder= r'C:\work_folder\Efficiency\test_data_RFB\new'
    output_folder = r'C:\work_folder\Efficiency\test_data_RFB\output'


    csv_files = find_csv_files(data_folder)
    all_data = []

    for file in csv_files:
        sn, freq, test_time, lot_number, rfb_details, dut_details, run_data = read_sensor_data(file)
        # convert test time to pandas datetime
        test_time = pd.to_datetime(test_time, format='%d-%b-%Y %H:%M:%S')
        for _, row in run_data.iterrows():
            data_entry = {
                'SerialNumber': sn,
                'Frequency': freq,
                'TestTime': test_time,
                'LotNumber': lot_number,
                **rfb_details,
                **dut_details,
                **row.to_dict()
            }
            all_data.append(data_entry)

    final_df = pd.DataFrame(all_data)
    print(final_df)
    # You can save final_df to a CSV or Excel file if needed
    # final_df.to_csv('compiled_sensor_data.csv', index=False)

    # Save final_df to an Excel file in the output folder
    final_df.to_excel(os.path.join(output_folder, 'compiled_sensor_data.xlsx'), index=False)


    





C:\work_folder\Efficiency\test_data_RFB\new\040685.csv
Found Run# data at line 2
C:\work_folder\Efficiency\test_data_RFB\new\040686.csv
Found Run# data at line 2
C:\work_folder\Efficiency\test_data_RFB\new\040687.csv
Found Run# data at line 2
  SerialNumber Frequency            TestTime LotNumber  \
0       040685  9.030000 2025-10-16 12:40:51     M4981   
1       040685  9.030000 2025-10-16 12:40:51     M4981   
2       040685  9.030000 2025-10-16 12:40:51     M4981   
3       040686  9.040000 2025-10-16 12:45:30     M4981   
4       040686  9.040000 2025-10-16 12:45:30     M4981   
5       040686  9.040000 2025-10-16 12:45:30     M4981   
6       040687  9.070000 2025-10-16 12:51:21     M4981   
7       040687  9.070000 2025-10-16 12:51:21     M4981   
8       040687  9.070000 2025-10-16 12:51:21     M4981   

                                           Filename: Software Version  \
0  O:\Transducers\Transducer Efficiency Testing\F...          2.0.9.0   
1  O:\Transducers\Transducer E

C:\Users\Santosh.Behera\AppData\Local\Temp\ipykernel_19416\1274995333.py:105: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  run_data = run_data.apply(pd.to_numeric, errors='ignore')
